In [1]:
!pip install nsepython

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from nsepython import *
import pandas as pd
from datetime import date

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
def my_equity_history_virgin(symbol,series,start_date,end_date):
    url="https://www.nseindia.com/api/historical/cm/equity?symbol="+symbol+"&series=[%22"+series+"%22]&from="+str(start_date)+"&to="+str(end_date)+""
    print("url is "+ url)
    payload = nsefetch(url)
    return pd.DataFrame.from_records(payload["data"])

In [4]:
my_equity_history_virgin("M_M", "EQ","21-05-2025", "11-06-2025") 

url is https://www.nseindia.com/api/historical/cm/equity?symbol=M_M&series=[%22EQ%22]&from=21-05-2025&to=11-06-2025


,_id,CH_SYMBOL,CH_SERIES,CH_MARKET_TYPE,CH_TIMESTAMP,TIMESTAMP,CH_TRADE_HIGH_PRICE,CH_TRADE_LOW_PRICE,CH_OPENING_PRICE,CH_CLOSING_PRICE,CH_LAST_TRADED_PRICE,CH_PREVIOUS_CLS_PRICE,CH_TOT_TRADED_QTY,CH_TOT_TRADED_VAL,CH_52WEEK_HIGH_PRICE,CH_52WEEK_LOW_PRICE,CH_TOTAL_TRADES,CH_ISIN,createdAt,updatedAt,__v,SLBMH_TOT_VAL,VWAP,mTIMESTAMP
0,68481c094e1df80b8ef258d7,M&M,EQ,N,2025-06-10,2025-06-09T18:30:00.000Z,3106.9,3059.8,3102.0,3067.5,3075.2,3087.4,1787253,5.510068e+09,3270.95,2425,111508,INE101A01026,2025-06-10T11:50:33.388Z,2025-06-10T11:50:33.388Z,0,None,3082.98,10-Jun-2025
1,6846ca89329f5654a9e4b0a5,M&M,EQ,N,2025-06-09,2025-06-08T18:30:00.000Z,3141.0,3078.0,3130.0,3087.4,3085.0,3106.5,1364639,4.233053e+09,3270.95,2425,76538,INE101A01026,2025-06-09T11:50:33.499Z,2025-06-09T11:50:33.499Z,0,None,3101.96,09-Jun-2025
2,6842d60a9946e2885cf61acb,M&M,EQ,N,2025-06-06,2025-06-05T18:30:00.000Z,3112.9,3027.0,3041.6,3106.5,3106.5,3041.6,2028071,6.237451e+09,3270.95,2425,95355,INE101A01026,2025-06-06T11:50:34.080Z,2025-06-06T11:50:34.080Z,0,None,3075.56,06-Jun-2025
3,684184899a6028178c29e34b,M&M,EQ,N,2025-06-05,2025-06-04T18:30:00.000Z,3095.0,2994.8,3078.0,3041.6,3045.0,3053.4,3504632,1.068703e+10,3270.95,2425,143954,INE101A01026,2025-06-05T11:50:33.239Z,2025-06-05T11:50:33.239Z,0,None,3049.40,05-Jun-2025
4,68403417f8805773fac6d069,M&M,EQ,N,2025-06-04,2025-06-03T18:30:00.000Z,3087.0,3040.1,3045.1,3053.4,3051.0,3046.5,1584653,4.851381e+09,3270.95,2425,82506,INE101A01026,2025-06-04T11:55:03.137Z,2025-06-04T11:55:03.137Z,0,None,3061.48,04-Jun-2025
5,683ee18950e76e71351c744b,M&M,EQ,N,2025-06-03,2025-06-02T18:30:00.000Z,3064.8,3027.0,3042.0,3046.5,3044.0,3025.9,3204094,9.759296e+09,3270.95,2425,139606,INE101A01026,2025-06-03T11:50:33.614Z,2025-06-03T11:50:33.614Z,0,None,3045.88,03-Jun-2025
6,683d90090c5070bb8d9304ad,M&M,EQ,N,2025-06-02,2025-06-01T18:30:00.000Z,3038.5,2962.3,2976.8,3025.9,3024.0,2976.8,2967741,8.948807e+09,3270.95,2425,170070,INE101A01026,2025-06-02T11:50:33.385Z,2025-06-02T11:50:33.385Z,0,None,3015.36,02-Jun-2025
7,68399dff96d3ed0d0efbdbe7,M&M,EQ,N,2025-05-30,2025-05-29T18:30:00.000Z,3017.4,2957.0,3005.0,2976.8,2971.0,3009.0,3700191,1.101632e+10,3270.95,2425,113144,INE101A01026,2025-05-30T12:01:03.023Z,2025-05-30T12:01:03.023Z,0,None,2977.23,30-May-2025
8,68384c6083e3b07d6ac5b287,M&M,EQ,N,2025-05-29,2025-05-28T18:30:00.000Z,3028.9,2987.2,2999.0,3009.0,3013.6,2997.8,2093429,6.287388e+09,3270.95,2425,102777,INE101A01026,2025-05-29T12:00:32.677Z,2025-05-29T12:00:32.677Z,0,None,3003.39,29-May-2025
9,6836fae518d296368fd00023,M&M,EQ,N,2025-05-28,2025-05-27T18:30:00.000Z,3044.8,2993.0,3039.7,2997.8,2994.5,3039.8,1491995,4.489415e+09,3270.95,2425,93521,INE101A01026,2025-05-28T12:00:37.622Z,2025-05-28T12:00:37.622Z,0,None,3009.00,28-May-2025


In [5]:
# You shall see beautiful use the logger function.
def my_equity_history(symbol,series,start_date,end_date):
    #We are getting the input in text. So it is being converted to Datetime object from String.
    start_date = datetime.datetime.strptime(start_date, "%d-%m-%Y")
    end_date = datetime.datetime.strptime(end_date, "%d-%m-%Y")
    print("Starting Date: "+str(start_date))
    print("Ending Date: "+str(end_date))

    #We are calculating the difference between the days
    diff = end_date-start_date
    print("Total Number of Days: "+str(diff.days))
    print("Total FOR Loops in the program: "+str(int(diff.days/40)))
    print("Remainder Loop: " + str(diff.days-(int(diff.days/40)*40)))


    total=pd.DataFrame()
    for i in range (0,int(diff.days/40)):

        temp_date = (start_date+datetime.timedelta(days=(40))).strftime("%d-%m-%Y")
        start_date = datetime.datetime.strftime(start_date, "%d-%m-%Y")

        print("Loop = "+str(i))
        print("====")
        print("Starting Date: "+str(start_date))
        print("Ending Date: "+str(temp_date))
        print("====")

        #total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
        #total=total.concat(equity_history_virgin(symbol,series,start_date,temp_date))
        total = pd.concat([total, my_equity_history_virgin(symbol, series, start_date, temp_date)])


        print("Length of the Table: "+ str(len(total)))

        #Preparation for the next loop
        start_date = datetime.datetime.strptime(temp_date, "%d-%m-%Y")


    start_date = datetime.datetime.strftime(start_date, "%d-%m-%Y")
    end_date = datetime.datetime.strftime(end_date, "%d-%m-%Y")

    print("End Loop")
    print("====")
    print("Starting Date: "+str(start_date))
    print("Ending Date: "+str(end_date))
    print("====")

    #total=total.append(equity_history_virgin(symbol,series,start_date,end_date))
    #total=total.concat(equity_history_virgin(symbol,series,start_date,end_date))
    total = pd.concat([total, my_equity_history_virgin(symbol, series, start_date, end_date)])


    print("Finale")
    print("Length of the Total Dataset: "+ str(len(total)))
    payload = total.iloc[::-1].reset_index(drop=True)
    return payload

In [6]:
import datetime

def fetch_equity_history_with_change(symbol, days, series="EQ"):
    end_date = datetime.datetime.now().strftime("%d-%m-%Y")
    start_date = (datetime.datetime.now() - datetime.timedelta(days=days)).strftime("%d-%m-%Y")
    print(start_date, end_date)
    df = my_equity_history(symbol, series, start_date, end_date)
    if df is None or df.empty:
        print(f"No data returned for {symbol}")
        return None           
    df["daily_change"] = df["CH_CLOSING_PRICE"].pct_change()
    # Uncomment below lines if you want to filter columns or drop first row
    # df = df[['CH_TIMESTAMP', 'daily_change']]
    # df = df.iloc[1:, :]
    #print(df)
    return df



In [7]:
fetch_equity_history_with_change("M_M", days=20, series="EQ")

22-05-2025 11-06-2025
Starting Date: 2025-05-22 00:00:00
Ending Date: 2025-06-11 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 22-05-2025
Ending Date: 11-06-2025
====
url is https://www.nseindia.com/api/historical/cm/equity?symbol=M_M&series=[%22EQ%22]&from=22-05-2025&to=11-06-2025
Finale
Length of the Total Dataset: 14


,_id,CH_SYMBOL,CH_SERIES,CH_MARKET_TYPE,CH_TIMESTAMP,TIMESTAMP,CH_TRADE_HIGH_PRICE,CH_TRADE_LOW_PRICE,CH_OPENING_PRICE,CH_CLOSING_PRICE,CH_LAST_TRADED_PRICE,CH_PREVIOUS_CLS_PRICE,CH_TOT_TRADED_QTY,CH_TOT_TRADED_VAL,CH_52WEEK_HIGH_PRICE,CH_52WEEK_LOW_PRICE,CH_TOTAL_TRADES,CH_ISIN,createdAt,updatedAt,__v,SLBMH_TOT_VAL,VWAP,mTIMESTAMP,daily_change
0,682f11e1fe6092ddd65ce662,M&M,EQ,N,2025-05-22,2025-05-21T18:30:00.000Z,3089.8,2980.5,3076.6,3006.3,3016.0,3090.7,3870830,1.165153e+10,3270.95,2425,187573,INE101A01026,2025-05-22T12:00:33.339Z,2025-05-22T12:00:33.339Z,0,None,3010.09,22-May-2025,NaN
1,68306362034b515579c7b193,M&M,EQ,N,2025-05-23,2025-05-22T18:30:00.000Z,3030.8,2984.2,3020.0,3012.7,3014.8,3006.3,1828093,5.504517e+09,3270.95,2425,105053,INE101A01026,2025-05-23T12:00:34.617Z,2025-05-23T12:00:34.617Z,0,None,3011.07,23-May-2025,0.002129
2,683457e1f73213993367d11e,M&M,EQ,N,2025-05-26,2025-05-25T18:30:00.000Z,3101.0,3027.7,3038.0,3080.2,3075.0,3012.7,2164190,6.663482e+09,3270.95,2425,150987,INE101A01026,2025-05-26T12:00:33.601Z,2025-05-26T12:00:33.601Z,0,None,3078.97,26-May-2025,0.022405
3,6835a96339be46f2e3d4e7f9,M&M,EQ,N,2025-05-27,2025-05-26T18:30:00.000Z,3085.0,3014.6,3080.2,3039.8,3042.0,3080.2,2599143,7.903217e+09,3270.95,2425,130075,INE101A01026,2025-05-27T12:00:35.603Z,2025-05-27T12:00:35.603Z,0,None,3040.70,27-May-2025,-0.013116
4,6836fae518d296368fd00023,M&M,EQ,N,2025-05-28,2025-05-27T18:30:00.000Z,3044.8,2993.0,3039.7,2997.8,2994.5,3039.8,1491995,4.489415e+09,3270.95,2425,93521,INE101A01026,2025-05-28T12:00:37.622Z,2025-05-28T12:00:37.622Z,0,None,3009.00,28-May-2025,-0.013817
5,68384c6083e3b07d6ac5b287,M&M,EQ,N,2025-05-29,2025-05-28T18:30:00.000Z,3028.9,2987.2,2999.0,3009.0,3013.6,2997.8,2093429,6.287388e+09,3270.95,2425,102777,INE101A01026,2025-05-29T12:00:32.677Z,2025-05-29T12:00:32.677Z,0,None,3003.39,29-May-2025,0.003736
6,68399dff96d3ed0d0efbdbe7,M&M,EQ,N,2025-05-30,2025-05-29T18:30:00.000Z,3017.4,2957.0,3005.0,2976.8,2971.0,3009.0,3700191,1.101632e+10,3270.95,2425,113144,INE101A01026,2025-05-30T12:01:03.023Z,2025-05-30T12:01:03.023Z,0,None,2977.23,30-May-2025,-0.010701
7,683d90090c5070bb8d9304ad,M&M,EQ,N,2025-06-02,2025-06-01T18:30:00.000Z,3038.5,2962.3,2976.8,3025.9,3024.0,2976.8,2967741,8.948807e+09,3270.95,2425,170070,INE101A01026,2025-06-02T11:50:33.385Z,2025-06-02T11:50:33.385Z,0,None,3015.36,02-Jun-2025,0.016494
8,683ee18950e76e71351c744b,M&M,EQ,N,2025-06-03,2025-06-02T18:30:00.000Z,3064.8,3027.0,3042.0,3046.5,3044.0,3025.9,3204094,9.759296e+09,3270.95,2425,139606,INE101A01026,2025-06-03T11:50:33.614Z,2025-06-03T11:50:33.614Z,0,None,3045.88,03-Jun-2025,0.006808
9,68403417f8805773fac6d069,M&M,EQ,N,2025-06-04,2025-06-03T18:30:00.000Z,3087.0,3040.1,3045.1,3053.4,3051.0,3046.5,1584653,4.851381e+09,3270.95,2425,82506,INE101A01026,2025-06-04T11:55:03.137Z,2025-06-04T11:55:03.137Z,0,None,3061.48,04-Jun-2025,0.002265


In [8]:
# nifty next 50
stocks = [
    "ADANIGREEN",
    "ADANIPORTS",
    "AMBUJACEM",
    "APOLLOHOSP",
    "AUROPHARMA",
    "BANDHANBNK",
    "BATAINDIA",
    "BERGEPAINT",
    "BIOCON",
    "BOSCHLTD",
    "CADILAHC",
    "COLPAL",
    "DABUR",
    "DIVISLAB",
    "DLF",
    "ESCORTS",
    "GAIL",
    "GLENMARK",
    "GODREJCP",
    "HAVELLS",
    "HDFCAMC",
    "HDFCLIFE",
    "HINDPETRO",
    "HINDZINC",
    "IBULHSGFIN",
    "ICICIGI",
    "ICICIPRULI",
    "IDFCFIRSTB",
    "INDIGO",
    "INDUSINDBK",
    "JINDALSTEL",
    "LTI",
    "LUPIN",
    "MCDOWELL-N",
    "MGL",
    "MRF",
    "NMDC",
    "PEL",
    "PIDILITIND",
    "PNB",
    "POWERGRID",
    "PVR",
    "RBLBANK",
    "SRF",
    "SRTRANSFIN",
    "SUNPHARMA",
    "TATACHEM",
    "TATACONSUM",
    "TATAMOTORS",
    "TATASTEEL",
    "TECHM",
    "TITAN",
    "UPL",
    "ULTRACEMCO",
    "WIPRO"
]

print(stocks)
#stocks = ['BAJFINANCE','JSWSTEEL','AXISBANK','M_M','ETERNAL','TATASTEEL','HINDALCO','TRENT','EICHERMOT','TITAN','HDFCBANK','ADANIENT','NTPC','BAJAJ-AUTO','COALINDIA','APOLLOHOSP','JIOFIN','INFY','ADANIPORTS','SBIN','ULTRACEMCO','TECHM','GRASIM','HINDUNILVR','TCS','ITC','POWERGRID','ICICIBANK','HCLTECH','LT','TATACONSUM','ASIANPAINT','RELIANCE']
#stocks = ['BAJFINANCE','JSWSTEEL','AXISBANK','M_M','ETERNAL']

['ADANIGREEN', 'ADANIPORTS', 'AMBUJACEM', 'APOLLOHOSP', 'AUROPHARMA', 'BANDHANBNK', 'BATAINDIA', 'BERGEPAINT', 'BIOCON', 'BOSCHLTD', 'CADILAHC', 'COLPAL', 'DABUR', 'DIVISLAB', 'DLF', 'ESCORTS', 'GAIL', 'GLENMARK', 'GODREJCP', 'HAVELLS', 'HDFCAMC', 'HDFCLIFE', 'HINDPETRO', 'HINDZINC', 'IBULHSGFIN', 'ICICIGI', 'ICICIPRULI', 'IDFCFIRSTB', 'INDIGO', 'INDUSINDBK', 'JINDALSTEL', 'LTI', 'LUPIN', 'MCDOWELL-N', 'MGL', 'MRF', 'NMDC', 'PEL', 'PIDILITIND', 'PNB', 'POWERGRID', 'PVR', 'RBLBANK', 'SRF', 'SRTRANSFIN', 'SUNPHARMA', 'TATACHEM', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO']


In [9]:
# 

In [10]:
#linear execution 
#from random import randint
# import pandas as pd
# import datetime

# # Initialize the final table

# total = pd.DataFrame()

# # Define time range

# for symbol in stocks:
#     print(f"Fetching data for: {symbol}")
#     try:
#      total= total+fetch_equity_history_with_change(symbol, days=365, series="EQ")
#     except Exception as err:
#         print(f"Error: Skipping symbol {symbol} due to error: {err}")

# # Final review
# print("Final dataset preview:")
# print(total.head())

# parallel execution

from random import randint
import pandas as pd
import datetime
import concurrent.futures

# Initialize the final table
total = pd.DataFrame()

def fetch_wrapper(symbol):
    print(f"Fetching data for: {symbol}")
    try:
        return fetch_equity_history_with_change(symbol, days=20, series="EQ")
    except Exception as err:
        print(f"Error: Skipping symbol {symbol} due to error: {err}")
        return pd.DataFrame()  # Return empty DataFrame on error

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(fetch_wrapper, stocks))

# Combine all results
total = pd.concat(results, ignore_index=True)

# Final review
print("Final dataset preview:")
print(total.head())

Fetching data for: ADANIGREEN
22-05-2025 11-06-2025
Starting Date: 2025-05-22 00:00:00
Ending Date: 2025-06-11 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 22-05-2025
Ending Date: 11-06-2025
====
url is https://www.nseindia.com/api/historical/cm/equity?symbol=ADANIGREEN&series=[%22EQ%22]&from=22-05-2025&to=11-06-2025
Fetching data for: ADANIPORTS
22-05-2025 11-06-2025
Starting Date: 2025-05-22 00:00:00
Ending Date: 2025-06-11 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 22-05-2025
Ending Date: 11-06-2025
====
url is https://www.nseindia.com/api/historical/cm/equity?symbol=ADANIPORTS&series=[%22EQ%22]&from=22-05-2025&to=11-06-2025
Fetching data for: AMBUJACEM
22-05-2025 11-06-2025
Starting Date: 2025-05-22 00:00:00
Ending Date: 2025-06-11 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting 

In [11]:
# Remove 'CH_' from column names
total.columns = [col.replace("CH_", "") for col in total.columns]
total.to_csv('../../inputdata/trading/stock_nifty_next50.csv',index=False)
#total.to_csv('./../inputdata/trading/total_may24_jun25.csv', sep='\t', index=False, header=True)
#payload.to_csv('./../inputdata/trading/total_may24_jun25_v2.csv', sep='\t', index=False, header=True)
#column_list=["CH_SYMBOL","CH_SERIES","CH_TIMESTAMP","TIMESTAMP","CH_TRADE_HIGH_PRICE","CH_TRADE_LOW_PRICE","CH_OPENING_PRICE","CH_CLOSING_PRICE","CH_LAST_TRADED_PRICE","CH_PREVIOUS_CLS_PRICE","CH_TOT_TRADED_QTY","CH_TOT_TRADED_VAL","CH_52WEEK_HIGH_PRICE","CH_52WEEK_LOW_PRICE","CH_TOTAL_TRADES","CH_ISIN","createdAt","updatedAt","SLBMH_TOT_VAL"]

#or use columns arg
#total.to_csv('../../inputdata/trading/total_niftyfifty.csv', columns = column_list,index=False)

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("../../inputdata/trading/stock_nifty_next50.csv")
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results = []

# Process each symbol separately
for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    
    # Calculate previous close and up days
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close'] # Up by at least 1% 
    sdf['Up_1'] = sdf['Up'].shift(1)
    
    # 10-day average volume
    sdf['Avg_Vol_10'] = sdf['TOT_TRADED_QTY'].rolling(window=10).mean()
    
    # Buy signal: 2 consecutive up closes + volume surge (>1.2x avg volume)
    sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10'])
  
    #sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10']) & (sdf['TIMESTAMP']== (datetime.datetime.now().date - datetime.timedelta(days=1)).date())
    # Filter out rows with current date
    # print ("timestoand and date")
    # print (sdf['TIMESTAMP'])
    # print ("today date")
    yesterday= ((datetime.datetime.now()- datetime.timedelta(days=1)).date())

    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        # Buy on the next day and hold for 3 days

        timestamp_date = pd.to_datetime(sdf.loc[i, 'TIMESTAMP']).date()
        total_traded_value = sdf.loc[i, 'TOT_TRADED_VAL']
        close_price = sdf.loc[i, 'CLOSING_PRICE']
        details = sdf.loc[i]
        # if(1==2):
        if(timestamp_date == yesterday)&(total_traded_value > 10000000)&(close_price > 50):
           # print(f"Skipping buy signal for {symbol} on {sdf.loc[i, 'TIMESTAMP']} as it is not yesterday's date {yesterday}")
            #print(f"Processing symbol: {symbol}, Buy signal at date {sdf.loc[i, 'TIMESTAMP']}")
            #results.append({'SYMBOL': symbol, 'DETAILS': details})
            results.append(details[['SYMBOL','CLOSING_PRICE', 'TIMESTAMP', 'TOT_TRADED_QTY', 'TOT_TRADED_VAL']])
    # else :  
    #     print(f"No trades for symbol {symbol}")
  
# if  len(results) == 0:
#     print("No profitable trades found.")
# else : 
    # Sort by total profit descending
results_df = pd.DataFrame(results)
results_df.to_csv('../../output/trading/stock_niftynext50_processed.csv', index=False)
print(results_df)  # Top 10 symbols by profit
print("Total trades found:", len(results_df))
# best = results_df.iloc[0]
#print(f"\nMost profitable symbol: {best['SYMBOL']} with total profit ₹{round(best['Total Profit (₹)'],2)} from {best['Trades']} trades")
#sum_profit = results_df['Total Profit (₹)'].sum()



        SYMBOL  CLOSING_PRICE  TIMESTAMP  TOT_TRADED_QTY  TOT_TRADED_VAL
13  ADANIGREEN        1060.50 2025-06-10         5441911    5.755703e+09
13   AMBUJACEM         561.10 2025-06-10         3239799    1.824607e+09
13      BIOCON         341.20 2025-06-10         6067297    2.086768e+09
13    HINDZINC         534.05 2025-06-10        11077729    5.959808e+09
13  INDUSINDBK         845.05 2025-06-10        10162134    8.620781e+09
13        LTIM        5385.00 2025-06-10          428695    2.306147e+09
13  TATAMOTORS         732.25 2025-06-10        19151889    1.395146e+10
13       TECHM        1610.90 2025-06-10         3324373    5.367190e+09
13       WIPRO         254.87 2025-06-10        10739851    2.732475e+09
Total trades found: 9
